# JoRo! Analytics

<center>

<hr>
<img src="http://i68.tinypic.com/16j4snn.jpg" alt="Golden State Warriors Team" width="800" height="900">
<hr>

<h2>"If you torture the data long enough, it will confess." <sub>- Ronald Coase, Economist</sub></h2>
<h2>"Never confuse motion with action." <sub>- Benjamin Franklin</sub></h2>

<hr>

<img src="http://i63.tinypic.com/2nk728y.png" alt="Portland Trail Blazers Logo" width="300" height="500">

</center>

###### Activate .ipynb Module

In [ ]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [ ]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [ ]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [ ]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [ ]:
mynba = find_notebook('mynba')

In [ ]:
path = '/Users/josephrosas/Documents/2education/2_py/finalproject/'

In [ ]:
NotebookFinder().find_module(mynba, path)

In [ ]:
sys.meta_path.append(NotebookFinder())

###### Import Modules

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import collections
import seaborn as sns
import plotly

In [ ]:
import mynba  # My Jupyter file (.ipynb) created as a module for this notebook.

# National Basketball Association (NBA) Analytics

<div align="justify">Here is my soft introduction to my NBA Analytics Python project. The datasets used in this project are results of web scraping through a popular Python module called <a href="http://selenium-python.readthedocs.io">Selenium</a>. I put this module to work on sports-related websites such as <a href="http://www.espn.com">ESPN.com</a> and <a href="https://stats.nba.com">Stats.NBA.com</a>. This web scraping resulted in the creation of three .ipynb files. The first one is <b>"NBA Web Scraping.ipynb"</b> - This file contains all foundational Python (and SQL) code I organized together in order to scrap relevant data from the aforementioned websites. The second is <b>"mynba.ipynb"</b> - This was created for the specific intent of providing a clean file with functions, models, CSV to pandas codes and it will continue growing the more insightful analtyics results we develop.
</div>

<hr>
<center><img src="http://i68.tinypic.com/2v10nz7.jpg" alt="San Antonio Spurs Players with mathematics as a background." width="600" height="700" ></center>
<hr>

<b>DataFrames</b> <p>(more to come)</p>
1. player_salaries
2. player_info
3. team_stats

<b>Classes</b> <p>(more to come)</p>
1. PlayerInfo
2. LeagueStats

<hr>
<b>References For Python / SQL / Query</b>
- https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_query.html#pandas.read_sql_query

<hr>

<b>References NBA Statistical Details</b>
- https://stats.nba.com/leaders/?Season=2017-18&SeasonType=Regular%20Season (PLAYERS)
- https://stats.nba.com/teams/traditional/?sort=W_PCT&dir=-1&Season=2017-18&SeasonType=Regular%20Season (TEAMS)

In [ ]:
pd.set_option('display.max_columns', None)  # Ability to view ALL columns

In [ ]:
# DataFrames from a previously created (mynba.ipynb) module - still a work in progress!

player_salaries = mynba.player_salaries
player_info = mynba.player_info
player_stats = mynba.player_stats
playerinfo = mynba.PlayerInfo
leaguestats = mynba.LeagueStats
teams = mynba.team_stats

In [ ]:
# Useful variables created during several exploratory stages

nba_rookies = player_info[player_info['EXP'] == 'R']

###### Example One [ Improving Deficiencies ] 

In [ ]:
# Sort (by number of wins) top 5 and last 5 NBA teams

top5 = teams.sort_values(by=['WIN'], ascending=False).head()
bot5 = teams.sort_values(by=['WIN'], ascending=True).head()

In [ ]:
# Index DataFrame to use relevant numerical data for comparison

first5 = top5.iloc[:, 6:]
last5 = bot5.iloc[:, 6:]

In [ ]:
# Dropping (removing) additional columns that do not fit the comparison objective

first5 = first5.drop(['PTS', 'BLKA', 'TOv'], axis=1)
last5 = last5.drop(['PTS', 'BLKA', 'TOv'], axis=1)

In [ ]:
# Find statistical average of best five (first5) and worst five (last5) teams

top_avg = first5.mean()
bot_avg = last5.mean()

In [ ]:
df = top_avg.subtract(
    bot_avg
)  # Subject from each other to see what categories are higher correlated

In [ ]:
df.plot(kind='bar', figsize=(12, 10), title='Average Difference')

In [ ]:
top5

In [ ]:
bot5

In [ ]:
top_avg[['3PA']] - bot_avg[['3PA']]

In [ ]:
top_avg[['REB']] - bot_avg[['REB']]

In [ ]:
suns_options = player_info[(player_info['3PA'] > 3.44) & (player_info['REB'] > 2.7)]

In [ ]:
suns_options[['NAME'],'EXP]]

###### Example Two